In [1]:
import numpy as np
np.random.seed(24)

In [2]:
import os
import torchaudio
import pandas as pd
import glob

In [3]:
import random
random.seed(24)

In [4]:
# torchaudio.datasets.LIBRISPEECH(root= , url = 'train-clean-100', download=True)

In [5]:
#f = open('/cas/DeepLearn/elperu/tmp/speech_datasets/LibriSpeech/README.TXT', 'r')
#readme = f.readlines()
#readme

## Dataset structure
100 hours of speech, different speakers reads chapters of books

### Analyze dataset

In [6]:
dataroot = '/cas/DeepLearn/elperu/tmp/speech_datasets/LibriSpeech/train-clean-100/'

In [7]:
speakers = os.listdir(dataroot)
print(f'Number of speaker: {len(speakers)}')

Number of speaker: 251


In [8]:
# Check speakers to keep
valid_spks = []   # spks with more than x utternaces
n_utterances_per_spk = 40
n_test_utterances_per_spk = 3

for speaker in speakers:
    chapters = os.listdir(os.path.join(dataroot, speaker))
    
    n_utters = 0
    
    for ch in chapters:
        n_utters += len([f for f in os.listdir(os.path.join(dataroot, speaker, ch)) if f.endswith('.flac')])
   
    if n_utters < n_utterances_per_spk + n_test_utterances_per_spk:
        print(f'Speaker {speaker} has {n_utters}')
    else: 
        valid_spks.append(speaker)
        
print('\nNumber of valid speakers: {}'.format(len(valid_spks)))

Speaker 7312 has 26

Number of valid speakers: 250


In [9]:
# Randomly choose N speakers for test set
N = 50

test_speakers = random.sample(valid_spks, N)

In [10]:
train_speakers = [s for s in valid_spks if s not in test_speakers]

In [11]:
len(train_speakers)

200

In [12]:
def config_dataframe(spk_list, train_flag=True):
    tot_rows = []
    train_test_rows = []   # files used to test the identification system
    
    for speaker in spk_list:
        chapters = os.listdir(os.path.join(dataroot, speaker))
    
        utterances = []
        for ch in chapters:
            utterances.extend(glob.glob(os.path.join(dataroot, speaker, ch, '*.flac')))

        for utter in utterances[:n_utterances_per_spk]:
            tot_rows.append([speaker, ch, os.path.abspath(os.path.join(dataroot, speaker, ch, utter))])
            
        if train_flag:
            for utter in utterances[n_utterances_per_spk: (n_utterances_per_spk + n_test_utterances_per_spk)]:
                train_test_rows.append([speaker, ch, os.path.abspath(os.path.join(dataroot, speaker, ch, utter))])
            
    assert len(tot_rows) == n_utterances_per_spk * len(spk_list)
    
    if train_flag:
        assert len(train_test_rows) == n_test_utterances_per_spk * len(spk_list)
    
    if not train_flag:
        
        return pd.DataFrame(tot_rows,  columns=['spk_id', 'ch_id', 'utter_path'])
    
    else:
        return pd.DataFrame(tot_rows,  columns=['spk_id', 'ch_id', 'utter_path']), pd.DataFrame(train_test_rows,  columns=['spk_id', 'ch_id', 'utter_path'])

In [13]:
df_test = config_dataframe(test_speakers, train_flag=False)

In [14]:
df_train, df_train_test = config_dataframe(train_speakers, train_flag=True)

In [16]:
# Store config files as csv
out_path = '/cas/DeepLearn/elperu/tmp/speech_datasets/LibriSpeech/'

In [18]:
df_test.to_csv(os.path.join(out_path, 'test_config.csv'), index=False)
df_train.to_csv(os.path.join(out_path, 'train_config.csv'), index=False)
df_train_test.to_csv(os.path.join(out_path, 'identification_test.csv'), index=False)